# Project: Title

Author and date

Notes:
* Running script using QIIME2 v2019.7
* Original script written by Roli Wilhelm

# Pipeline to process raw sequences into phyloseq object with DADA2 ###
* Prep for Import to QIIME2  (Combine two index files)
* Import to QIIME2
* Demultiplex
* Denoise and Merge
* Prepare OTU Tables and Rep Sequences  *(Note: sample names starting with a digit will break this step)*
* Classify Seqs

*100% Appropriated from the "Atacama Desert Tutorial" for QIIME2*

### Pipeline can handle both 16S rRNA gene and ITS sequences####
* Tested on 515f and 806r
* Tested on ITS1

### Commands to install dependencies ####
##### || QIIME2 ||
** Note: QIIME2 is still actively in development, and I've noticed frequent new releases. Check for the most up-to-date conda install file <https://docs.qiime2.org/2017.11/install/native/#install-qiime-2-within-a-conda-environment>

* wget https://data.qiime2.org/distro/core/qiime2-2018.2-py35-linux-conda.yml
* conda env create -n qiime2-2018.2 --file qiime2-2018.2-py35-linux-conda.yml
* source activate qiime2-pipeline

##### || Copyrighter rrn database ||
* The script will automatically install the curated GreenGenes rrn attribute database
* https://github.com/fangly/AmpliCopyrighter

##### || rpy2 (don't use conda version) ||
* pip install rpy2  

##### || phyloseq ||
* conda install -c r r-igraph 
* Rscript -e "source('http://bioconductor.org/biocLite.R');biocLite('phyloseq')" 

##### || R packages ||
* ape   (natively installed in conda environment)

### Citations ###
* Caporaso, J. G., Kuczynski, J., Stombaugh, J., Bittinger, K., Bushman, F. D., Costello, E. K., *et al.* (2010). QIIME allows analysis of high-throughput community sequencing data. Nature methods, 7(5), 335-336.

* McMurdie and Holmes (2013) phyloseq: An R Package for Reproducible Interactive Analysis and Graphics of Microbiome Census Data. PLoS ONE. 8(4):e61217

* Paradis E., Claude J. & Strimmer K. 2004. APE: analyses of phylogenetics and evolution in R language. Bioinformatics 20: 289-290.

* Angly, F. E., Dennis, P. G., Skarshewski, A., Vanwonterghem, I., Hugenholtz, P., & Tyson, G. W. (2014). CopyRighter: a rapid tool for improving the accuracy of microbial community profiles through lineage-specific gene copy number correction. Microbiome, 2(1), 11.


# Step 1: User Input

Notes
* The metadata file must be located in the project directory
* Metadata file must be .tsv format and have one column named "BarcodeSequence" with the relevant primers (rev. comp reverse concatenated with forward primer sequence)
* The output directories will be created inside the project directory

In [ ]:
import os, re, numpy as np

# Prepare an object with the name of the library and all related file paths
# datasets = [['name', 'project directory path', 'output directory name', 'raw data directory', read1 file name, read2 file name, 'metadata file name','domain of life'],...]
datasets = [[]]
# Notes: 
# It's no longer possible to install QIIME1. Read1 and Read2 were modified with seqid_truncate.py as a work around
# seqid_truncate.py removes a portion of each sequence ID that isn't useful but interferes with demultiplexing
# This made the fastq files compatible with our custom barcode extraction script (concatenate_barcodes_mod.py) that replaces QIIME1's (extract_barcodes.py)

# Set # of processors to use
processors = 10

# Which bacterial database will you use? Silva or GreenGenes
db = "Silva"

# Phylogenetic tree (non-fungal data)
treepath = "" # full path to the phylogenetic tree file to be made
treename = "" # name prefix for the tree file

## Enter minimum support for keeping QIIME classification
# Note: Classifications that do not meet this criteria will simply be retained, but labeled 'putative'
min_support = 0.9

# Step 2: Concatenate barcodes for QIIME2 pipeline

In [ ]:
## Note: QIIME takes a single barcode file. The command 'extract_barcodes.py' concatenates the forward and reverse read barcode and attributes it to a single read.

# See http://qiime.org/tutorials/processing_illumina_data.html

for dataset in datasets:
    directory = dataset[1]
    output = dataset[2]
    raw = dataset[3]
    read1 = dataset[4]
    read2 = dataset[5]
    #index1 = dataset[6]
    #index2 = dataset[7]
    
    # Run extract_barcodes to merge the two index files
    #!python2 /opt/anaconda2/bin/extract_barcodes.py --input_type barcode_paired_end -f $raw/$index1 -r $raw/$index2 --bc1_len 8 --bc2_len 8 -o $directory/$output
    
    # Note:
    # Above (commented out) step is broken because QIIME1 cannot be reinstalled
    # Instead, I used concatenate_barcodes_mod.py script outside this pipeline to produce barcodes.fastq
    # I'll work on incorporating custom script into future versions of pipeline
    
    # QIIME2 import requires a directory containing files names: forward.fastq.gz, reverse.fastq.gz and barcodes.fastq.gz 
    !ln -s $raw/$read1 $directory/$output/forward.fastq.gz
    !ln -s $raw/$read2 $directory/$output/reverse.fastq.gz
    
    # Gzip the barcodes files (apparently necessary)
    !pigz -p 5 $directory/$output/barcodes.fastq

    # Remove orphaned reads files (not needed)
    !rm $directory/$output/reads?.fastq


# Step 3: Import into QIIME2

In [ ]:
for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    output = dataset[2]
    
    os.system(' '.join([
        "qiime tools import",
        "--type EMPPairedEndSequences",
        "--input-path "+directory+"/"+output,
        "--output-path "+directory+"/"+output+"/"+name+".qza"
    ]))
    
    # This more direct command is broken by the fact QIIME uses multiple dashes in their arguments (is my theory)
    #!qiime tools import --type EMPPairedEndSequences --input-path $directory/output --output-path $directory/output/$name.qza
     

# Step 4: Demultiplex

Notes
* The barcode you supply to QIIME is now a concatenation of your forward and reverse barcode.
* Your 'forward' barcode is actually the reverse complement of your reverse barcode and the 'reverse' is your forward barcode. The file 'primers.complete.csv' provides this information corresponding to the Buckley Lab 'primer number'
* This quirk could be corrected in how different sequencing facilities pre-process the output from the sequencer

In [ ]:
### SLOW STEP (~ 2 - 4 hrs)

for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    output = dataset[2]
    metadata = dataset[6]
    
    os.system(' '.join([
        "qiime demux emp-paired",
        "--m-barcodes-file "+directory+"/"+metadata,
        "--m-barcodes-column BarcodeSequence",
        "--p-no-golay-error-correction",
        "--i-seqs "+directory+"/"+output+"/"+name+".qza",
        "--o-per-sample-sequences "+directory+"/"+output+"/"+name+".demux",
        "--o-error-correction-details "+directory+"/"+output+"/"+name+".demux-details.qza"
    ]))
    
    # This more direct command is broken by the fact QIIME uses multiple dashes in their arguments (is my theory)
    #!qiime demux emp-paired --m-barcodes-file $directory/$metadata --m-barcodes-category BarcodeSequence --i-seqs $directory/output/$name.qza --o-per-sample-sequences $directory/output/$name.demux
    

# Step 5: Visualize quality scores and determine trimming parameters

Based on the graph produced using the following command enter the trim and truncate values. Trim refers to the start of a sequence and truncate the total length (i.e. number of bases to remove from end)

Drop output from below command into https://view.qiime2.org

In [ ]:
for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    output = dataset[2]
    
    os.system(' '.join([
        "qiime demux summarize",
        "--i-data "+directory+"/"+output+"/"+name+".demux.qza",
        "--o-visualization "+directory+"/"+output+"/"+name+".demux.qzv"
    ]))

# Step 6: Trimming parameters | USER INPUT REQUIRED

Notes
* All trimming parameters must be the same for datasets that will be directly compared to one-another because ASVs are determined by sequence length

In [ ]:
## User Input Required

## Input your trimming parameters into a python dictionary for all libraries
# trim_dict["LibraryName1"] = [trim_forward, truncate_forward, trim_reverse, truncate_reverse]
# trim_dict["LibraryName2"] = [trim_forward, truncate_forward, trim_reverse, truncate_reverse]

# The example in the Atacam Desert Tutorial trims 13 bp from the start of each read and does not remove any bases from the end of the 150 bp reads:
#  --p-trim-left-f 13 \  
#  --p-trim-left-r 13 \
#  --p-trunc-len-f 150 \
#  --p-trunc-len-r 150

trim_dict = {}

trim_dict["xxx"] = [1,1,1,1] # input values here

# Step 7: Trim, denoise and join (aka 'merge') reads using DADA2

In [ ]:
### SLOW STEP (~ 6 - 8 hrs, IF multithreading is used)

for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    output = dataset[2]
    
    os.system(' '.join([
        "qiime dada2 denoise-paired",
        "--i-demultiplexed-seqs "+directory+"/"+output+"/"+name+".demux.qza",
        "--o-table "+directory+"/"+output+"/"+name+".table.qza",
        "--o-representative-sequences "+directory+"/"+output+"/"+name+".rep-seqs.qza",
        "--o-denoising-stats "+directory+"/"+output+"/"+name+".denoising-stats.qza",
        "--p-trim-left-f "+str(trim_dict[name][0]),
        "--p-trim-left-r "+str(trim_dict[name][2]),
        "--p-trunc-len-f "+str(trim_dict[name][1]),
        "--p-trunc-len-r "+str(trim_dict[name][3]),
        "--p-n-threads",
        str(processors)
    ]))

# Step 8: Create summary of ASVs

In [ ]:
for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    output = dataset[2]
    metadata = dataset[6]
    
    os.system(' '.join([
        "qiime feature-table summarize",
        "--i-table "+directory+"/"+output+"/"+name+".table.qza",
        "--o-visualization "+directory+"/"+output+"/"+name+".table.qzv",
        "--m-sample-metadata-file "+directory+"/"+metadata
    ]))

    os.system(' '.join([
        "qiime feature-table tabulate-seqs",
        "--i-data "+directory+"/"+output+"/"+name+".rep-seqs.qza",
        "--o-visualization "+directory+"/"+output+"/"+name+".rep-seqs.qzv"
    ])) 

# Step 9: Classify sequences

Using SILVA database v128

In [ ]:
### Example of creating Silva Classifier DB  (very slow: use a pre-built one if possible)

#### For Silva, remove all the alignment information (unsure of the impact of keeping it) using the following python code:

# ```python 
# import re
# from Bio import SeqIO

# output = open("silva.nr_v128.fasta", "w")

# for record in SeqIO.parse(open("silva.nr_v128.align", "rU"), "fasta") :
#     seq = str(record.seq)
#     seq = re.sub("\.|-","",seq)  # Remove "." and "-"

#     output.write(">"+record.id+"\n"+seq+"\n")```

# #### Import fasta sequence file and taxonomy file as .qza
# ```bash
# qiime tools import
#   --type 'FeatureData[Sequence]'
#   --input-path silva.nr_v128.fasta
#   --output-path silva.nr_v128.qza

# qiime tools import 
#   --type 'FeatureData[Taxonomy]' 
#   --source-format HeaderlessTSVTaxonomyFormat 
#   --input-path silva.nr_v128.tax 
#   --output-path silva.nr_v128.taxonomy.qza```

# #### Run QIIME2 'fit-classifier-naive-bayes'
# ```bash
# qiime feature-classifier fit-classifier-naive-bayes 
#   --i-reference-reads silva.nr_v128.qza 
#   --i-reference-taxonomy silva.nr_v128.taxonomy.qza 
#   --o-classifier silva.nr_v128.nb.classifier.qza```


In [ ]:
## Note: Different QIIME2 versions can conflict with previously donwloaded databases. This section might have to be updated.
try:
    if db == "GreenGenes":
        classification_db = "/home/db/GreenGenes/qiime2_13.8.99_515.806_nb.classifier.qza"
    else:
        classification_db = "/home/cassi/databases/silva_v132_classifier100819/silva.nr_v132qiime2.2019.7.classifier.qza"
        
except:
        classification_db = "/home/cassi/databases/silva_v132_classifier100819/silva.nr_v132qiime2.2019.7.classifier.qza"
        
for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    output = dataset[2]
    metadata = dataset[6]
    domain = dataset[7]

    # Classify
    if domain == 'bacteria':
        print(' '.join([
            "qiime feature-classifier classify-sklearn",
            "--i-classifier",
            classification_db,
            "--i-reads "+directory+"/"+output+"/"+name+".rep-seqs.qza",
            "--o-classification "+directory+"/"+output+"/"+name+".taxonomy.qza",
            "--p-n-jobs",
            str(processors)
        ]))

    if domain == 'fungi':
        os.system(' '.join([
            "qiime feature-classifier classify-sklearn",
            "--i-classifier /home/db/UNITE/qiime2_unite_ver7.99_20.11.2016_classifier.qza",
            "--i-reads "+directory+"/"+output+"/"+name+".rep-seqs.qza",
            "--o-classification "+directory+"/"+output+"/"+name+".taxonomy.qza",
            "--p-n-jobs",
            str(processors)
        ]))

    # Output summary
    print(' '.join([
        "qiime metadata tabulate",
        "--m-input-file "+directory+"/"+output+"/"+name+".taxonomy.qza",
        "--o-visualization "+directory+"/"+output+"/"+name+".taxonomy-summary.qzv"
    ])) 

# Step 10: Merge representative sequences for phylogenetic tree

If dataset is split into multiple libraries, one tree should be created using all data

In [ ]:
repseqslist = []

for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    output = dataset[2]
    domain = dataset[6]
    
    if domain != "fungi":
        repseqslist.append("--i-data " + os.path.join(directory, output, name+".rep-seqs.qza"))
        
    #print(repseqslist)

os.system(' '.join([
    "qiime feature-table merge-seqs",
    " ".join(repseqslist),
    "--o-merged-data "+treepath+"/"+treename+".rep-seqs-merged.qza"
]))

# Step 11: Make phylogenetic tree (16S only)

In [ ]:
for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    output = dataset[2]
    domain = dataset[6]

    # Generate alignment with MAFFT
    os.system(' '.join([
        "qiime alignment mafft",
        "--i-sequences "+treepath+"/"+treename+".rep-seqs-merged.qza",
        "--o-alignment "+treepath+"/"+treename+".rep-seqs-merged-aligned.qza",
        "--p-n-threads",
        str(processors)
    ]))
    
    # Mask hypervariable regions in alignment
    os.system(' '.join([
        "qiime alignment mask",
        "--i-alignment "+treepath+"/"+treename+".rep-seqs-merged-aligned.qza",
        "--o-masked-alignment "+treepath+"/"+treename+".rep-seqs-merged-aligned-masked.qza",
    ]))
    
    # Generate tree with FastTree
    os.system(' '.join([
        "qiime phylogeny fasttree",
        "--i-alignment "+treepath+"/"+treename+".rep-seqs-merged-aligned-masked.qza",
        "--o-tree "+treepath+"/"+treename+".tree-unrooted.qza",
        "--p-n-threads",
        str(processors)
    ]))
    
    # Root the tree
    os.system(' '.join([
        "qiime phylogeny midpoint-root",
        "--i-tree "+treepath+"/"+treename+".tree-unrooted.qza",
        "--o-rooted-tree "+treepath+"/"+treename+".tree-rooted.qza"
    ]))

# Step 12: Reformat taxonomy

In [ ]:
## Make function to reformat taxonomy file to contain full column information 
# and factor in the confidence of the taxonomic assignment
# modified to include species level identification, no longer differentiates databases, may cause problems for non-Silva db?

def format_taxonomy(tax_file, min_support):
    output = open(re.sub(".tsv","-fixed.tsv",tax_file), "w")
    
    full_rank_length = 7
    output.write("\t".join(["OTU","Domain","Phylum","Class","Order","Family","Genus","Species"])+"\n")

    with open(tax_file, "r") as f:
        next(f)

        for line in f:
                line = line.strip()
                line = line.split("\t")

                read_id = line[0]
                tax_string = line[1]

                # Split full rank into ranks
                full_rank = tax_string.split(";")

                ## Identify the lowest classified taxonomic rank
                # Account for cases when a taxonomic rank contains an empty space (common in GreenGenes output)
                last_classified = full_rank[len(full_rank)-1]            

                count = 1
                while last_classified == " ":
                    last_classified = full_rank[len(full_rank)-count]
                    count = count + 1

                # Annotate the last classified as 'putative' if it does not meet the minimum support criteria
                if float(line[2]) < float(min_support):
                        full_rank[full_rank.index(last_classified)] = "putative "+last_classified
                        last_classified = "putative "+last_classified

                #print(last_classified)
                # Chloroplast, didn't change because confidence > 0.9

                # Add in columns containing unclassified taxonomic information
                for n in range(full_rank.index(last_classified)+1, full_rank_length, 1):               
                    try:
                        full_rank[n] = "unclassified "+last_classified
                    except:
                        full_rank.append("unclassified "+last_classified)

                # Write taxonomy to file
                output.write(read_id+"\t"+'\t'.join(full_rank)+"\n")
                
    return()

# Step 13: Export from QIIME2

In [ ]:
for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    output = dataset[2]
    metadata = dataset[6]

    # Final output names
    fasta_file = directory+"/"+output+"/final/"+name+".rep-seqs-final.fasta"
    tax_file = directory+"/"+output+"/final/"+name+".taxonomy-final.tsv"
    count_table = directory+"/"+output+"/final/"+name+".counts-final.biom"
    tax_tree = treepath+"/"+treename+".tree-final.nwk"
    
    # Make final data directories
    if not os.path.isdir(os.path.join(directory, output, "final")):
        !mkdir $directory/$output/final

    with open(os.path.join(directory, output, name+"repseqslist"), "w") as outfile:
        for i in repseqslist:
            outfile.write(i+"\n")

    # Export classifications
    os.system(' '.join([
        "qiime tools export",
        "--input-path "+directory+"/"+output+"/"+name+".taxonomy.qza",
        "--output-path "+directory+"/"+output+"/"
    ]))
    
    # Reformat classifications to meet phyloseq format   
    format_taxonomy(directory+"/"+output+"/taxonomy.tsv", min_support)

    # Export ESV table
    os.system(' '.join([
        "qiime tools export",
        "--input-path "+directory+"/"+output+"/"+name+".table.qza",
        "--output-path "+directory+"/"+output+"/"
    ]))

    # Export ESV sequences
    os.system(' '.join([
        "qiime tools export",
        "--input-path "+directory+"/"+output+"/"+name+".rep-seqs.qza",
        "--output-path "+directory+"/"+output+"/"
    ]))
    
    # Rename exported files
    %mv $directory/$output/dna-sequences.fasta $fasta_file
    %mv $directory/$output/feature-table.biom $count_table
    %mv $directory/$output/taxonomy-fixed.tsv $tax_file
    
# Export tree
os.system(' '.join([
    "qiime tools export",
    "--input-path "+treepath+"/"+treename+".tree-rooted.qza",
    "--output-path "+treepath+"/"
]))

# Rename tree file
%mv $treepath/$treename+.tree.nwk $tree_file

# Done!